In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from WF_SDK import device
from WF_SDK import scope
from WF_SDK import wavegen

%matplotlib inline

In [ ]:
class ADSHardware():
    """Class of functions for interfacing with the ADS.
    """

    def __init__(self):
        self.handle = None

    def startup(self):
        """Connects to the ADS. Defines 'handle', the address to the ADS.
        Must be run at the beginning of every program using the ADS.
        """
        self.handle = device.open()

    def open_scope(self, buffer_size=1000, sample_freq=100e6):
        """Opens connection to the scope.

        Args:
            buffer_size (int, optional): How many data points are temporarily stored
            before being returned. The buffer is a temporary slot for storing a small amount of
            data before it is transferred to its final destination. Defaults to 1000.
            sample_freq (int, optional): How frequently the oscilloscope will sample
            from the input. Defaults to 100e6. You can decrease this if you have too
            many data points/the function is taking awhile to run for the time scale you need.
            (1e6 or even 16e3 can be reasonable selections.)
        """
        scope.open(self.handle, buffer_size=buffer_size, sampling_frequency=sample_freq)

    def trigger_scope(self, channel=1, level=0.1):
        """Sets trigger level for the scope. Kind of a buggy function; not used.

        Args:
            channel (int, optional): Selects which channel of scope to read out. 
            Defaults to 1.
            level (float, optional): Sets trigger level for scope. Defaults to 0.1.
        """
        scope.trigger(self.handle, enable=True, source=scope.trigger_source.analog, channel=channel,
                      edge_rising=True, level=level)

    def read_scope(self, channel=1):
        """Collects data from the scope.

        Args:
            channel (int, optional): Which channel to read from. Defaults to 1.

        Returns:
            buffer (array): An array of output data points. The buffer is a temporary slot 
            for storing a small amount of data before it is transferred to its final destination.
        """
        buffer = scope.record(self.handle, channel=channel)
        return buffer

    def close_scope(self):
        """Closes connection to the scope.
        """
        scope.close(self.handle)

    def use_wavegen(self, channel=1, function=wavegen.function.sine, offset_v=0, freq_hz=1e3, amp_v=1):
        """Runs the wavegen producing function with given parameters.

        Args:
            channel (int, optional): Which channel output is at. Defaults to 1.
            function (function object, optional): What type of function to output. 
            Defaults to wavegen.function.sine.
            offset (int, optional): Voltage offset (V). Defaults to 0.
            freq (int, optional): Frequency (Hz). Defaults to 1e3.
            amp (int, optional): Amplitude (V). Defaults to 1.
        """
        wavegen.generate(self.handle, channel=channel, function=function, offset=offset_v,
                         frequency=freq_hz, amplitude=amp_v)

    def close_wavegen(self):
        """Closes wavegen.
        """
        wavegen.close(self.handle)

    def disconnect(self):
        """Closes ADS connection. Must be run at the end of every program.
        """
        device.close(self.handle)

In [ ]:
def oscilloscope_run(ads_object: ADSHardware, n_points: int, channel: int, sampling_freq=1e6):
    """Collects data from the oscilloscope.

    Args:
        ads_object (ADSHardware object): the ADS being used
        n_points (int): number of data points to collect; 1000-2000 is a good starting point
        channel (int): which channel to collect data from
        sampling_freq (int, optional): How frequently the oscilloscope will sample
        from the input. Defaults to 1e6. You can decrease this if you have too
        many data points/the function is taking awhile to run for the time scale you need.
        (16e3 can be a reasonable selection.)

    Returns:
        data (dict): has two keys, "x" and "y" which have time (ms) and voltage (V) data
    """
    #test 16 khz, 1 mhz as well for sampling_freq
    data = {}
    ads_object.open_scope(sample_freq=sampling_freq)
    buffer = ads_object.read_scope(channel=channel)
    data["y"] = np.mean(buffer)
    data["x"] = np.array([0])
    for i in range(n_points):
        buffer = np.mean(ads_object.read_scope(channel=channel))
        data["y"] = np.append(data["y"], buffer)
        #MODIFY THE LINE BELOW THIS ONE IN L10.2(d)
        data["x"] = np.append(data["x"], np.array([i]))
    ads_object.close_scope()
    return data

In [ ]:
def fft(data: dict):
    """Takes an FFT of input data.

    Args:
        data (dict): Provides x data in ms and y data in V obtained from oscilloscope.
    Returns:
        fft_result (dict): a dictionary with two keys, "frequencies" and "magnitudes",
                            containing the frequencies and magnitudes from the FFT.
    """
    fft_result = {}
    #FILL IN THIS FUNCTION FOR L10.3(b) and L10.3(c)

    fft_result["frequencies"] = ...
    fft_result["magnitudes"] = ...

    return fft_result

In [ ]:
wavegen_functions = {"sine":wavegen.function.sine, "square":wavegen.function.square,
                     "triangle":wavegen.function.triangle}

In [ ]:
ads = ADSHardware()
ads.startup()

In [ ]:

ads.use_wavegen(channel=1, function=wavegen_functions["sine"], offset_v=0, freq_hz=1e3, amp_v=1)
### FILL IN THIS LINE FOR L10.2(a)
raw_data = ...
ads.close_wavegen()

In [ ]:
### UNCOMMENT THIS CODE FOR L10.3(a)
#fft_data = fft(raw_data)

In [ ]:
### UNCOMMENT THIS CODE FOR L10.2(b)
#plt.plot(raw_data["x"], raw_data["y"])
#plt.xlabel('Time (ms)')
#plt.ylabel('Voltage (V)')
#plt.show()

In [ ]:
### PLOT YOUR DATA HERE FOR L10.3(d)

In [ ]:
#Only run this cell if your code errored in the middle of oscilloscope_run()
ads.close_scope()

In [ ]:
#Run this cell at the end of every test
ads.close_wavegen()
ads.disconnect()